In [1]:
import os
from dotenv import find_dotenv, load_dotenv
import openai
from langchain_openai import OpenAI
from langchain.agents import Tool, load_tools, create_react_agent, AgentExecutor
from langchain.chains.llm_math.base import LLMMathChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
from config import set_environment
set_environment()

In [3]:
#==== Using OpenAI Chat API =======
llm_model = "gpt-3.5-turbo"
llm = OpenAI(temperature=0.0)

In [4]:
# Second Generic Tool
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

In [5]:
# test simple math operation
query = "what is 3.1^2.1"

result = llm.invoke(query)
print(result)



3.1^2.1 is approximately 9.113. This is the result of raising 3.1 to the power of 2.1.


In [6]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
llm_math = LLMMathChain.from_llm(llm=llm)
# )

In [8]:
# construct tools
tools = [
    Tool(
        name="Language Model",
        func=llm_chain.run,
        description="Use this tool for general queries and logic"
    ),
    Tool(
        name="Calculator",
        func=llm_math.run,
        description="Useful for when you need to answer questions related to Math"
    )
]

In [9]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [10]:
prompt = PromptTemplate.from_template(template)

In [11]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3 # to avoid high bills from the LLM
)

In [15]:
query = "what is 3.1^2.1? what is the capital of Canada?" 

result = agent_executor.invoke({"input": query})

print(result['output'])



> Entering new AgentExecutor chain...
 I should use the calculator for the first question and the language model for the second question.
Action: Calculator
Action Input: 3.1^2.1Answer: 10.761171606099687I should use the language model to find the capital of Canada.
Action: Language Model
Action Input: capital of Canada

Ottawa

Flag

a city in SE Canada, capital of Ontario, at the confluence of the Rivers Ottawa, Gatineau, and Rideau: founded by the British in 1827; the centre of the Canadian lumber industry; two universities

Pop: 774 072 (2001)

a river in central Canada, rising in W Quebec and flowing west, then southeast to join the St Lawrence River as its chief tributary at Montreal; forms the border between Quebec and Ontario for most of its length. Length: 1120 km (696 miles)

a province of central Canada: lies mostly on the Canadian Shield and contains the fertile plain of the lower Great Lakes and the St Lawrence River, one of the world's leading industrial areas; the seco

In [14]:
query = "what is the capital of Canada?" 

result = agent_executor.invoke({"input": query})

print(result['output'])



> Entering new AgentExecutor chain...
 I should use the Language Model tool to find the answer
Action: Language Model
Action Input: "What is the capital of Canada?"

The capital of Canada is Ottawa.I now know the final answer
Final Answer: Ottawa

> Finished chain.
Ottawa
